# 02_ml_pipeline

This notebook covers the following tasks:

1. Build the Pipeline
2. Compile the Pipeline
3. Execute the Pipeline on Vertex AI

## Setup

### Import libraries

In [1]:
%load_ext autotime

time: 479 µs (started: 2022-11-27 20:23:09 +00:00)


In [2]:
import os
import datetime
import json
import kfp
from kfp.v2.dsl import component
from kfp.v2 import dsl, compiler
from google.cloud import aiplatform as aip
from google_cloud_pipeline_components import aiplatform as gcc_aip
import gcsfs
import importlib

time: 2.98 s (started: 2022-11-27 20:23:09 +00:00)


### Set configurations and constants

In [3]:
NOTEBOOK = '02_ml_pipeline'
REGION = "us-west1"
PROJECT = 'babyweight-prediction'
BUCKET = 'b_w_bucket'
BQ_DATASET = "bw_dataset"
APPNAME = "bw-prediction"

os.environ["REGION"] = REGION
os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET

time: 807 µs (started: 2022-11-27 20:23:12 +00:00)


### Set service account access for Vertex AI Pipeline

In [4]:
SERVICE_ACCOUNT = "69318036822-compute@developer.gserviceaccount.com"

time: 1.44 ms (started: 2022-11-27 20:23:12 +00:00)


In [5]:
!gcloud config list

[core]
account = 69318036822-compute@developer.gserviceaccount.com
disable_usage_reporting = True
project = babyweight-prediction

Your active configuration is: [default]
time: 934 ms (started: 2022-11-27 20:23:12 +00:00)


In [6]:
!gcloud auth list

                  Credentialed Accounts
ACTIVE  ACCOUNT
*       69318036822-compute@developer.gserviceaccount.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`

time: 930 ms (started: 2022-11-27 20:23:13 +00:00)


### Initialize the Vertex AI SDK & Tensorboard Instance

In [7]:
aip.init(project=PROJECT, location=REGION, staging_bucket=BUCKET)

time: 655 µs (started: 2022-11-27 20:23:14 +00:00)


In [8]:
## Create a Tensorboard instance
#tb = aip.Tensorboard.create(display_name=f"{APPNAME}_{NOTEBOOK}", description=NOTEBOOK)
#tb.resource_name

time: 1.49 ms (started: 2022-11-27 20:23:14 +00:00)


## 1. Construct the Pipeline

### 1.1. Define pipeline constants

In [9]:
#TIMESTAMP = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
PIPELINE_URI = f"gs://{BUCKET}/{APPNAME}"

#Data: extract_source_data
SOURCE_BQ_TABLE_ID = "bigquery-public-data.samples.natality"
YEAR = 2003
EXTRACTED_BQ_TABLE_ID = f"{PROJECT}.{BQ_DATASET}.{APPNAME}_extracted_{YEAR}"

#Data: prepped_table_creating_task
LIMIT = 500000 #changed to 500,000
PREPPED_BQ_TABLE_ID = f"{PROJECT}.{BQ_DATASET}.{APPNAME}_extracted_{YEAR}_limit_{LIMIT}" 
VERTEX_DATASET_DISPLAY_NAME = f"{APPNAME}_extracted_{YEAR}_limit_{LIMIT}_dataset"

## bqml
BQML_MODEL_NAME = f"bqml_{APPNAME}"
BQML_MODEL_VERSION_ALIASES = BQML_MODEL_NAME
VAR_TARGET = "weight_pounds"
#
##automl
AUTOML_MODEL_NAME = f"automl_{APPNAME}"
COLUMN_SPECS = {
    "is_male": "categorical",
    "mother_age": "numeric",
    "plurality": "categorical",
    "gestation_weeks": "numeric",
    "cigarette_use": "categorical",
    "alcohol_use": "categorical",
}
#
##model validation
REFERENCE_METRIC_NAME = "mae"
THRESHOLDS_DICT = {"mae": 50.0}
#
##deployment
ENDPOINT_DISPLAY_NAME = f"{APPNAME}_endpoint"
DEPLOY_IMAGE="us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-7:latest"
DEPLOY_MACHINE_TYPE = "n1-highmem-4"
#DEPLOY_MACHINE_TYPE = "n1-standard-1"
DEPLOY_MIN_REPLICA_COUNT = 1
DEPLOY_MAX_REPLICA_COUNT = 1

#EXPLANATION_PARAMATERS = {"sampledShapleyAttribution": {"pathCount": 10}}
#EXPLANATION_METADATA_JSON_PATH = f"{CUSTOM_TRAININGS_URI[5:]}/explanation_metadata.json"


time: 4.4 ms (started: 2022-11-27 20:23:14 +00:00)


### 1.3. BQML queries

In [10]:
from src.pipeline import data_comp
from src.pipeline import bqml_comp
from src.pipeline import automl_comp
from src.pipeline import model_validation_comp
from src.pipeline import serving_validation_comp
#
importlib.reload(data_comp)
importlib.reload(bqml_comp)
importlib.reload(automl_comp)
importlib.reload(model_validation_comp)
importlib.reload(serving_validation_comp)

<module 'src.pipeline.serving_validation_comp' from '/home/jupyter/babyweight-vertex-ai/src/pipeline/serving_validation_comp.py'>

time: 85.6 ms (started: 2022-11-27 20:23:14 +00:00)


In [11]:
QUERY_BUILDING_BQML_MODEL = bqml_comp.create_query_build_bqml_model(
    project=PROJECT,
    bq_dataset=BQ_DATASET,
    bq_model_name=BQML_MODEL_NAME,
    bq_version_aliases=BQML_MODEL_VERSION_ALIASES,
    var_target=VAR_TARGET,
    bq_train_table_id=PREPPED_BQ_TABLE_ID
)
print("QUERY_BUILDING_BQML_MODEL:", QUERY_BUILDING_BQML_MODEL)

QUERY_BUILDING_BQML_MODEL: 
    CREATE MODEL IF NOT EXISTS `babyweight-prediction.bw_dataset.bqml_bw-prediction`
    OPTIONS(
        model_type = 'DNN_LINEAR_COMBINED_REGRESSOR',
        model_registry = 'vertex_ai', 
        vertex_ai_model_version_aliases = ['bqml_bw-prediction'],
        input_label_cols = ['weight_pounds'],
        data_split_col = 'custom_splits',
        data_split_method = 'CUSTOM',
        HIDDEN_UNITS = [256, 128, 64],
        OPTIMIZER = 'adagrad',
        BATCH_SIZE = HPARAM_CANDIDATES([16, 32, 64]),
        DROPOUT =  HPARAM_CANDIDATES([0, 0.1, 0.2]),
        MAX_ITERATIONS = 5,
        MAX_PARALLEL_TRIALS = 5,
        NUM_TRIALS = 20
        ) AS
    SELECT * EXCEPT(splits),
        CASE
            WHEN splits = 'VALIDATE' THEN 'EVAL'
            ELSE splits
        END AS custom_splits
    FROM `babyweight-prediction.bw_dataset.bw-prediction_extracted_2003_limit_500000`
    
time: 803 µs (started: 2022-11-27 20:23:14 +00:00)


### 1.4. Define the pipeline

In [12]:
@dsl.pipeline(name = APPNAME, pipeline_root = PIPELINE_URI)
def vertex_ai_pipeline(
    service_account: str=SERVICE_ACCOUNT,
    project: str=PROJECT,
    region: str=REGION,
):
    from google_cloud_pipeline_components import aiplatform as gcpc_aip
    from google_cloud_pipeline_components.v1.bigquery import BigqueryCreateModelJobOp
    from google_cloud_pipeline_components.v1.bigquery import BigqueryEvaluateModelJobOp
    
    ##################################
    #extract data from original natality dataset
    extracted_source_data_task = data_comp.extract_source_data(
        project=project,
        region=region,
        year=YEAR,
        in_bq_table_id=SOURCE_BQ_TABLE_ID, 
        out_bq_table_id=EXTRACTED_BQ_TABLE_ID,  
    ).set_display_name('create_extracted_table') 
    
    ##################################
    
    prepped_table_creating_task = data_comp.preprocess_data(
        project=project,
        region=region,
        limit=LIMIT,
        in_bq_table_id=EXTRACTED_BQ_TABLE_ID, 
        out_bq_table_id=PREPPED_BQ_TABLE_ID,  
    ).set_display_name('create_prepped_table').after(extracted_source_data_task)
    
    
    vertex_dataset_creating_task = gcc_aip.TabularDatasetCreateOp(
        project=project,
        display_name=VERTEX_DATASET_DISPLAY_NAME,
        bq_source=prepped_table_creating_task.outputs["out_bq_table_uri"],
    ).set_display_name('create_vertex_dataset')    

    ##################################
    
    bqml_creating_task = BigqueryCreateModelJobOp(
        project=project, 
        location="US",
        query=QUERY_BUILDING_BQML_MODEL,
    ).set_display_name('create_bqml_model').after(prepped_table_creating_task)
    
    bqml_evaluating_task = BigqueryEvaluateModelJobOp(
        project=project, 
        location="US",
        model=bqml_creating_task.outputs["model"]
    ).set_display_name('evaluate_bqml_model')
    
    bqml_evaluation_metrics_collecting_task = bqml_comp.collect_eval_metrics_bqml(
        eval_metrics_artifact=bqml_evaluating_task.outputs["evaluation_metrics"]
    )

    ###################################
    
    automl_creating_task = gcc_aip.AutoMLTabularTrainingJobRunOp(
        project=project,
        display_name=f"{AUTOML_MODEL_NAME}_TrainingJob",
        optimization_prediction_type="regression",
        optimization_objective="minimize-rmse",
        budget_milli_node_hours=3000, #changed from 1,000
        model_display_name=AUTOML_MODEL_NAME,
        dataset=vertex_dataset_creating_task.outputs["dataset"],
        column_specs=COLUMN_SPECS,
        target_column=VAR_TARGET,
    ).set_display_name('create_automl_model')
    
    automl_evaluation_metrics_collecting_task = automl_comp.collect_eval_metrics_automl(
        region='us-central1', 
        model=automl_creating_task.outputs["model"]
    )
    
    ###################################
    
    best_model_task = model_validation_comp.select_best_model(
        automl_metrics=automl_evaluation_metrics_collecting_task.outputs["metrics"],
        automl_model=automl_creating_task.outputs["model"],
        bqml_metrics=bqml_evaluation_metrics_collecting_task.outputs["metrics"],
        bqml_model=bqml_creating_task.outputs["model"],  #Argument type "Model" is incompatible with the input type "google.VertexModel"
        reference_metric_name=REFERENCE_METRIC_NAME,
        thresholds_dict=THRESHOLDS_DICT,
    )
    
    ##### DEPLOYMENT ############################### 
    endpoint_creating_task = gcc_aip.EndpointCreateOp(
        project=project,
        location='us-central1',
        #location=region,
        display_name=ENDPOINT_DISPLAY_NAME,
    ).set_display_name('create_endpoint')

    #===============================#    
    # If the deploy condition is True, then deploy the best model.
    
    with dsl.Condition(
        best_model_task.outputs["deploy_decision"] == "true",
        name="deploy_decision",
    ):
               
        #===============================#
        with dsl.Condition(
            best_model_task.outputs["best_model_name"] == "automl",
            name="deploy_automl",
        ): 
        
            # deploy the model to endpoint
            automl_model_deploy_task = gcc_aip.ModelDeployOp(
                #model=best_model_task.outputs["best_model"],
                model=automl_creating_task.outputs["model"],
                endpoint=endpoint_creating_task.outputs["endpoint"],
                dedicated_resources_machine_type=DEPLOY_MACHINE_TYPE,
                dedicated_resources_min_replica_count=DEPLOY_MIN_REPLICA_COUNT,
                dedicated_resources_max_replica_count=DEPLOY_MAX_REPLICA_COUNT,
                traffic_split={"0": 100},
            ).set_display_name('deploy_model_to_endpoint')    
            
    
       
    

time: 4.79 ms (started: 2022-11-27 20:23:14 +00:00)


## 2. Compile the Pipeline

In [13]:
# Create a new (local) directory to store the complied file
DIR = f"temp/{NOTEBOOK}"
!rm -rf {DIR}
!mkdir -p {DIR}

time: 244 ms (started: 2022-11-27 20:23:14 +00:00)


In [14]:
compiled_package = f"{DIR}/compiled_package.json"

compiler.Compiler().compile(
    pipeline_func = vertex_ai_pipeline,
    package_path = compiled_package
)

/opt/conda/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1293: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


time: 679 ms (started: 2022-11-27 20:23:14 +00:00)


## 3. Execute the Pipeline on Vertex AI

### 3.1. Create Vertex AI Pipeline Job

In [15]:
pipeline_job = aip.PipelineJob(
    display_name = f"{APPNAME}",
    template_path = compiled_package,
    pipeline_root=PIPELINE_URI,
    parameter_values = {
        "project": PROJECT,
        "region": REGION,
    },
    #enable_caching = False,        #//TRUE, by default//
    labels = {
        'notebook':f'{NOTEBOOK}',
        'app':f'{APPNAME}'}
)

time: 179 ms (started: 2022-11-27 20:23:15 +00:00)


### 3.2. Run the job

In [ ]:
response = pipeline_job.run(
    service_account = SERVICE_ACCOUNT,
    #sync = True
)

Creating PipelineJob
PipelineJob created. Resource name: projects/69318036822/locations/us-west1/pipelineJobs/bw-prediction-20221127202315
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/69318036822/locations/us-west1/pipelineJobs/bw-prediction-20221127202315')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-west1/pipelines/runs/bw-prediction-20221127202315?project=69318036822
PipelineJob projects/69318036822/locations/us-west1/pipelineJobs/bw-prediction-20221127202315 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/69318036822/locations/us-west1/pipelineJobs/bw-prediction-20221127202315 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/69318036822/locations/us-west1/pipelineJobs/bw-prediction-20221127202315 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/69318036822/locations/us-west1/pipelineJobs/bw-prediction-20221127202315 current state

## 4. View the Pipeline results

In [ ]:
pipeline_df = aip.get_pipeline_df(APPNAME)
pipeline_df = pipeline_df[pipeline_df.pipeline_name == APPNAME]
pipeline_df

In [ ]:
PROJECT_NUMBER = pipeline_job.resource_name.split("/")[1]
PIPELINE_JOB_ID = pipeline_job.name
PIPELINE_TASKS_URI = f"{PIPELINE_URI}/{PROJECT_NUMBER}/{PIPELINE_JOB_ID}"

print("PROJECT_NUMBER:",PROJECT_NUMBER)
print("PIPELINE_JOB_ID:",PIPELINE_JOB_ID)
print("PIPELINE_TASKS_URI:",PIPELINE_TASKS_URI)

In [ ]:
all_pipeline_tasks = pipeline_job.gca_resource.job_detail.task_details
endpoint_create_task_index = 0 
print("All pipeline tasks:")
for i in range (len(all_pipeline_tasks)):
    print(f" {i}: {all_pipeline_tasks[i].task_name}")
    if all_pipeline_tasks[i].task_name =="endpoint-create": endpoint_create_task_index = i 

print("--- Index of `endpoint-create` task:", endpoint_create_task_index)

In [ ]:
endpoint_resource_name = all_pipeline_tasks[endpoint_create_task_index].outputs["endpoint"].artifacts[0].metadata["resourceName"]
endpoint_resource_name

***Click here to verify:***  
https://console.cloud.google.com/vertex-ai/locations/us-central1/endpoints/5840539796048445440?project=asc-ahnat-adsi-sandbox